In [1]:
import numpy as np

phi = np.loadtxt('/content/train.csv', dtype='float', delimiter=',', skiprows=1,
                 usecols=tuple(range(1, 14)))

In [2]:
y = np.loadtxt('/content/train.csv', dtype='float', delimiter=',', skiprows=1,
               usecols=14, ndmin=2)

In [3]:
phi_tst = np.loadtxt('/content/test.csv', dtype='float', delimiter=',',
                      skiprows=1, usecols=tuple(range(1, 14)))


In [4]:
phi_tst = np.concatenate((phi_tst, np.ones((105, 1))), axis=1)
phi = np.concatenate((phi, np.ones((400, 1))), axis=1)

In [5]:
for i in range(0, 13):
    cl_mx = max(phi[:, i])
    cl_mn = min(phi[:, i])
    phi[:, i] = (phi[:, i] - cl_mn) / (cl_mx - cl_mn)
    phi_tst[:, i] = (phi_tst[:, i] - cl_mn) / (cl_mx - cl_mn)

In [6]:
y = np.log(y)

def del_w(p, phi, w):
    if p == 2:
        delw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        delw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return delw

In [7]:
fnames = {'otpt_file.csv': 2.0,
             'otptfile_p1.csv': 1.75,
             'otptfile_p2.csv': 1.5,
             'otptfile_p3.csv': 1.3
             }

In [8]:
for (fname, p) in fnames.items():
    
    w = np.zeros((14, 1))

    lambd = 0.2

    t = 0.00012

    w_new = w - t * del_w(p, phi, w)

    i = 0
    
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * del_w(p, phi, w)
        i = i + 1

    id_test = np.loadtxt('/content/test.csv', dtype='int', delimiter=',',skiprows=1, usecols=0, ndmin=2)

    y_test = np.exp(np.dot(phi_tst, w_new))

    np.savetxt(fname, np.concatenate((id_test, y_test), axis=1), delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')